In [1]:
import sys
sys.path.append('../src/')

import pandas as pd
import requests
import numpy as np
import geocoder
import data_io
import time
import tokens

import utils

In [2]:
#locations that failed to geocode but could be filled by hand
WEIRD_LOC_DICT = {'westseneca':'west seneca, ny',
                 'linolakes': 'lino lakes, mn',
                 'newportrichey':'new port richey, fl',
                 'mayslick': 'mays lick, ky',
                 'boiseid': 'boise, id',
                 'dixonil': 'dixon, il',
                 'dallastx': 'dallas, tx',
                 '5779purpleleafcourtfrederickmd': 'frederick, md',
                 '07060northplainfieldnj': 'north plainfield, nj',
                 '08108haddontownshipnj': 'haddon township, nj',
                 '33134coralgablesfl': 'coral gables, fl',
                 '92587canyonlakeca': 'canyon lake, ca',
                 '95682cameronparkca': 'cameron park, ca',
                 '92075solanabeachca': 'solana beach, ca',
                 'de forest, wi': 'deforest, wi',
                 'horsesho, tx': 'horseshoe, tx',
                 'wesley c, fl': 'wesley chapel, fl'}

CENSUS_KEY = tokens.CENSUS_KEY
MB_TOKENS = tokens.MAPBOX_TOKENS

STATE_DICT = dict((v,k) for k,v in utils.us_state_abbrev.items())
CLEANED_FOLDER_NAME = 'mapbox_geocode'
NOT_FOUND_FILENAME = 'locs_not_found_master_recode_v2.csv'

In [3]:
import string
def contains_only_digits(s):
    # True for "", "0", "123"
    # False for "1.2", "1,2", "-1", "a", "a1"
    if type(s) != str:
        return True
    for ch in s:
        if not ch in string.digits:
            return False
    return True    

def has_numbers(inputString):
     return any(char.isdigit() for char in inputString)


def clean_location(x, other_loc, state):
    if contains_only_digits(other_loc) == False:
        if type(state) == str:
            state = state.lower()
        for k in WEIRD_LOC_DICT.keys():
            if k in x:
                x = WEIRD_LOC_DICT[k]
                return x
        if has_numbers(x) == True:
            x = ''.join(c for c in x if not c.isnumeric())
            if ' - ' in x:
                x = x.replace(" - ", "")
            if '-' in x:
                x = x.replace("-", "")
        if x[-4:] == ', us':
            x = x[0:len(x)-4]
        if ',' not in x and type(state)==str:
            comma_loc = x.find(state)
            old = x
            start = old[0:comma_loc]
            print(old, other_loc, state)
            try:
                if start[-1] == ' ':
                    start = start[0:len(start)-1]
                x = start +', '+old[comma_loc:]
                print(x)
            except:
                x = 'none'
            
        if 'a f b' in x:
            x = x.replace('a f b', 'air force base')
        if 'mc ' in x:
            if 'mc ,' not in x:
                x = x.replace('mc ', 'mc')
        if x[0]==' ':
            x = x[1:]
        
        
    return x

def clean_other_loc(x, other_loc, state):
    if contains_only_digits(other_loc) == False:
        if type(x) == str:
            if type(state) == str:

                state = state.lower()

            for k in WEIRD_LOC_DICT.keys():
                if k in x:
                    x = WEIRD_LOC_DICT[k]
            if has_numbers(x) == True:
                x = ''.join(c for c in x if not c.isnumeric())
                if ' - ' in x:
                    x = x.replace(" - ", "")
                if '-' in x:
                    x = x.replace("-", "")
            if x[-4:] == ', us':
                x = x[0:len(x)-4]
            if ',' not in x and type(state)==str:
                comma_loc = x.find(state)
                old = x
                start = old[0:comma_loc]
                print(old, other_loc, state)
                try:
                    if start[-1] == ' ':
                        start = start[0:len(start)-1]
                    x = start +', '+old[comma_loc:]
                    print(x)
                except:
                    x = 'none'

            if 'a f b' in x:
                x = x.replace('a f b', 'air force base')
            if 'mc ' in x:
                if 'mc ,' not in x:
                    x = x.replace('mc ', 'mc')
            if x[0]==' ':
                x = x[1:]
            
            x = x[0:len(x)-2]
        
        
    return x


In [4]:
def get_token(index):
    tokens =  MB_TOKENS
    if index % 2 == 0:
        return tokens[0]
    else:
        return tokens[1]
    
'''
GEOCODER FUNCTION
'''

def __coords_to_fip(lat,lng):
    result_d = {'search_status':'',
             'state_county_fips_str':np.nan,
             'county_name':np.nan}
    try:
        #get FIP code for corresponding lat/long
        fcc_req = 'https://geo.fcc.gov/api/census/block/find?'\
        +f'latitude={lat}&longitude={lng}&showall=false&format=json&key={CENSUS_KEY}'
        resp = requests.get(fcc_req)
        result = resp.json()
    except Exception as e:
        msg = 'error in fip retrieval: ' + str(e)
        print(msg)
        result_d['search_status'] = result_d['search_status'] + msg + '; '

    try:
        state_county_fips = result['County']['FIPS']
        county = result['County']['name']
        result_d['state_county_fips_str'] = state_county_fips
        result_d['county_name'] = county
    except Exception as e:
        msg = 'error in fip retrieval: ' + str(e)
        result_d['search_status'] = result_d['search_status'] + msg 
    
    return result_d
                            
def assign_county_fips(df, min_index = 0):
    start_time = time.time()
    #not_found_df = pd.DataFrame()
    
    for i,(idx,row) in enumerate(df.iterrows()):
        if not pd.isnull(row.state_county_fips_str): 
            # this row is geocoded already
            continue 
        time.sleep(0.5)
        token = get_token(i)
        location = row['location_tosearch']
        state_full = row['location_statefullname']
        
        print(location)
        
        # limit search results to US districts,place and locality
        # this is very helpful so MapBox doesnt match with the wrong place
        g = geocoder.mapbox(location, key=token, country='US',types='district,place,locality')
        df.loc[idx,'search_status'] = g.status
        
        g = g.json
        
        if g is None:
            print('nothing returned!')
        else:
            if g['status'] == 'OK':
                if 'state' in g.keys():
                    if g['state'] == state_full and g['country'] == 'United States':
                        lat = g['lat']
                        lng = g['lng']
                        df.loc[idx, 'location_lat'] = lat
                        df.loc[idx, 'location_lng'] = lng
                        result_d = __coords_to_fip(lat,lng)
                        for k,v in result_d.items(): df.loc[idx,k] = v
            else:
                print('status issue: ', g)

        if i % 100 == 0 and i > min_index:
            print(i)
            curr_time = time.time()
            print('min since start: ', (curr_time-start_time)/60)
            print('saving...')
            df.to_csv(data_io.input_cleaned/'geolocations'/f'unique_locations_w_fips_scraping.csv',
                                     encoding='utf-8')
    return df
    
    

In [5]:
fname = 'unique_locations_from_US_cancer_campaigns_2018_2021.csv'
df = pd.read_csv(data_io.input_cleaned/'geolocations'/fname,index_col=[0])

In [6]:
df.head()

,location_city,location_stateprefix
campaign_id,,
5cf2pqo,"Dayton, OH",OH
6wxxj-steve-mcallister,"Lilburn, GA",GA
HelptheShoeDoctor,"San Leandro, CA",CA
HopeforlifeSharith,"Revere, MA",MA
LeslieCarey,"Wilmington, DE",DE


Construct column to be used for searching, only for first time searching

In [20]:
rescrape = True
if not rescrape:
    df['location_statefullname'] = df.location_stateprefix.map(STATE_DICT)
    print(df.location_statefullname.isna().sum())
    df["location_city_only"]  = df.location_city.str.split(',').apply(
        lambda x: x[0]).str.strip()
    # prefer searching with city, fullstate_name over city, state_abbreviation
    df['location_tosearch'] = df.location_city.where(
            df.location_statefullname.isna(),
            df.location_city_only + ', ' + df.location_statefullname)

    df['state_county_fips_str'] = np.nan
    df['county_name'] = np.nan
    df['search_status'] = ''

0


Geocode each unique location in df using column location_tosearch

In [22]:
outputs = assign_county_fips(df)
output_path = data_io.input_cleaned/'geolocations'/f'unique_locations_w_fips_scraped.csv'
outputs.to_csv(output_path,encoding='utf-8')

Dayton, Ohio
Lilburn, Georgia
San Leandro, California
Revere, Massachusetts
Wilmington, Delaware
Island Lake, Illinois
McAllen, Texas
Lodi, New Jersey
Palm City, Florida
Chesapeake, Virginia
Fort Walton Beach, Florida
Oswego, New York
Scottsdale, Arizona
Aberdeen, South Dakota
Fort Lauderdale, Florida
Santa Barbara, California
Olympia, Washington
Altus, Oklahoma
Sunnyvale, Texas
Wright City, Missouri
Whitestone, New York
Mykawa, Texas
Fort Worth, Texas
Marlton, New Jersey
Dublin, Ohio
Aiea, Hawaii
Lakeside Park, Kentucky
Tigard, Oregon
Tomball, Texas
Paw Paw, West Virginia
Houston, Texas
San Clemente, California
Washington, Indiana
Overland Park, Kansas
New Cumberland, Pennsylvania
Aurora, Colorado
Lincoln, Nebraska
Summit, Illinois
Myrtle Beach, South Carolina
Pulaski, New York
Salt Lake City, Utah
Vienna, Virginia
Pamela Heights, Texas
Philadelphia, Pennsylvania
Holland, Michigan
Chicago, Illinois
Tucson, Arizona
Braintree, Massachusetts
Joshua Tree, California
Washington Township, P

400
min since start:  9.232905117670695
saving...
Manchester Township, Michigan
Kissimmee, Florida
Pembroke, North Carolina
Wilson, North Carolina
Lansing, Michigan
O’Fallon, Missouri
Absecon, New Jersey
Battle Ground, Washington
Huntington, Vermont
Saint Helens, Oregon
Coopersville, Michigan
Troy, Missouri
Buffalo, New York
Wilsonville, Oregon
Springfield, Oregon
Pasadena, California
Hampton, Virginia
Oatfield, Oregon
Thousand Oaks, California
Albuquerque, New Mexico
San Luis, Arizona
Olney, Maryland
League City, Texas
Fort Branch, Indiana
Odessa, Florida
Modesto, California
Jacksonville, Florida
Arroyo Seco, New Mexico
Sanford, Florida
Westville, Indiana
Morrow, Ohio
Cypress, Texas
Old Bridge Township, New Jersey
South San Francisco, California
Winton, California
New Tripoli, Pennsylvania
Providence, Rhode Island
Cibolo, Texas
Omaha, Nebraska
Hawthorne, California
Wantagh, New York
Midland, Texas
Eagan, Minnesota
Dawsonville, Georgia
Holland, New York
Vista, California
Fond du Lac, W

Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/Waimea/Kamuela CDP, Hawaii.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/Waimea/Kamuela%20CDP,%20Hawaii.json?access_token=pk.eyJ1IjoiaGlsbHRydWUxMSIsImEiOiJja240eDh1MzYwMXozMndsOGkzZzcxa2VpIn0.tc-OseoKMoTTRLCRKP0W8A&country=US&types=district%2Cplace%2Clocality


nothing returned!
Conway, South Carolina
Hawaii National Park, Hawaii
Kalispell, Montana
South Charleston, West Virginia
Rochester, Washington
Salem, Massachusetts
Plainview, Minnesota
Tampa, Florida
Winston-Salem, North Carolina
Vacaville, California
Grove City, Ohio
Benton Hills, North Carolina
Merrifield, Minnesota
Peoria, Illinois
Malden, Massachusetts
Horace, North Dakota
Leesville, Louisiana
Knightdale, North Carolina
Carpinteria, California
Butler, New Jersey
Great Falls, Montana
Baton Rouge, Louisiana
La Habra, California
Chapel Hill, North Carolina
Glendale, Arizona
Kirkwood, Missouri
Haverford Township, Pennsylvania
St. Charles, Illinois
Evansville, Indiana
Bend, Oregon
Locust Grove, Oklahoma
Lakewood, California
Saint Paul, Minnesota
Sarasota, Florida
Cave Creek, Arizona
Pearl, Mississippi
Monroe, Georgia
Satellite Beach, Florida
Bemidji, Minnesota
Havelock, North Carolina
Winthrop, Maine
Virginia Beach, Virginia
Carpentersville, Illinois
Forest Lake, Minnesota
Andover, Minn

Sicklerville, New Jersey
Oakes, North Dakota
Massapequa Park, New York
Grand Island, New York
Boonsboro, Maryland
Springfield, Missouri
Metairie, Louisiana
Cumming, Georgia
Germantown, Maryland
Nacogdoches, Texas
Devine, Texas
Castorland, New York
McPherson, Kansas
Ashland, Missouri
Blairstown, New Jersey
Lockport, New York
Jackson, Wyoming
Oxnard, California
Winchester, Virginia
Albany, New York
Indian Wells, Arizona
Rockford, Michigan
Gilroy, California
Folsom, Louisiana
Bremen, Maine
Port Orange, Florida
Covington, Georgia
Santa Clara, California
Altamonte Springs, Florida
Pasadena, Maryland
Lawrence, Kansas
Sand Springs, Oklahoma
Anaheim, California
Upton, Massachusetts
Pell City, Alabama
Splendora, Texas
Palmdale, California
Shirley, New York
Waterbury, Connecticut
Beaverdam Lake-Salisbury Mills, New York
Lewis Center, Ohio
Lunenburg, Massachusetts
Washington, Kansas
Lake Orion, Michigan
Fairfax, California
Temecula, California
Braemar Tartan Hills Parkway, Virginia
Branson, Misso

...

Walton, Kentucky
Springfield, South Carolina
Honey Brook, Pennsylvania
Jasonville, Indiana
Nunica, Michigan
Washington Court House, Ohio
Industry, Ohio
Flowood, Mississippi
Webster, Minnesota
West Branch, Iowa
Henrico, North Carolina
Greeley, Kansas
Inola, Oklahoma
Farmersville, California
Point Arena, California
Emmett, Idaho
Bridgewater, Vermont
Capitol Heights, Maryland
Ridgely, Maryland
Greenville, Illinois
Burke, New York
Morris, New York
Morriston, Florida
Versailles, Indiana
Max Meadows, Virginia
Arlington, Kansas
Biglerville, Pennsylvania
Medway, Ohio
Sharon, Wisconsin
Canal Fulton, Ohio
Indian Wells, California
Sunnyside, Arizona
Radcliffe, Iowa
Edgemere, Maryland
Wagener, South Carolina
Village of Lake Isabella, Michigan
Abingdon, Illinois
Honeymoon Island, Virginia
Crows Landing, California
Catawissa, Pennsylvania
Sandisfield, Massachusetts
Iron River, Michigan
Massena, New York
Palmersville, Tennessee
Brogden, North Carolina
Bonifay, Florida
10700
min since start:  172.6468

Milford, Kansas
Wilmot, New Hampshire
China, Texas
Hastings, Florida
Bunkie, Louisiana
Arkoma, Oklahoma
Milford, Indiana
Tulare, South Dakota
Summerville, Louisiana
Kasota, Minnesota
Alvin, Wisconsin
Port Orford, Oregon
Girard, Kansas
Valley Falls, New York
Cochran, Georgia
Oakville, Iowa
Lakota, North Dakota
Esty, West Virginia
Broylesville, Tennessee
Saint Regis, Montana
Minden City, Michigan
Metropolis, Illinois
Herrick Center, Pennsylvania
Decorah, Iowa
Dushore, Pennsylvania
Piermont, New York
Fabens, Texas
Dunsmuir, California
West Branch, Michigan
Drewryville, Virginia
Wittenberg, Wisconsin
Hawley, Texas
Bay City, Wisconsin
Deerwood, Minnesota
Mount Ayr, Iowa
Lake Medina Highlands, Texas
Hebron, Maine
Lake Alfred, Florida
Glennville, Georgia
Benwood, West Virginia
Dittmer, Missouri
Blair, Oklahoma
Huntington, Texas
Forestdale, Massachusetts
Joseph, Oregon
Pana, Illinois
Woodcliff Lake, New Jersey
Lauderdale, Mississippi
11100
min since start:  178.6447131196658
saving...
Lucedale

West Point, New York
Princess Anne, Maryland
Dundee, New York
Dighton, Massachusetts
Cascade, Virginia
Wellfleet, Massachusetts
Scott Air Force Base, Illinois
Glenwood, Maryland
Sunburg, Minnesota
Harleton, Texas
Butler, Alabama
Ogunquit, Maine
Clearwater, Kansas
Teasdale, Utah
New Providence, New Jersey
Rivesville, West Virginia
Troy, New Hampshire
Mount Jackson, Virginia
Shannon, Georgia
Glengary, West Virginia
Welaka, Florida
Greenville, Mississippi
Blauvelt, New York
Carlock, Illinois
Schuyler, Nebraska
Monroe, Utah
Hampton, Iowa
Golf View Manor, Florida
Oak View, California
French Settlement, Louisiana
Kelayres, Pennsylvania
Piedmont, Alabama
Augusta, Arkansas
Isle, Minnesota
Nicasio, California
Thayne, Wyoming
Woodruff, Arizona
Ferriday, Louisiana
Waretown, New Jersey
Pulaski, Virginia
Burlington, Kansas
Glouster, Ohio
Cygnet, Ohio
Eagle Creek, Oregon
Bruceville-Eddy, Texas
Hudson, North Carolina
Frenchburg, Kentucky
Red Cliff, Colorado
North Whitehall Township, Pennsylvania
Ecke

Status code 403 from https://api.mapbox.com/geocoding/v5/mapbox.places/Coaldale;Six Mile Run Post Office (historical), Pennsylvania.json: ERROR - 403 Client Error: Forbidden for url: https://api.mapbox.com/geocoding/v5/mapbox.places/Coaldale;Six%20Mile%20Run%20Post%20Office%20(historical),%20Pennsylvania.json?access_token=pk.eyJ1IjoiaHF0cnVvbmcxMSIsImEiOiJja240eDZjdDAxbHkxMnZtaDJpODJzbGthIn0.tD6DehvVJVsDRP2A2VM8dw&country=US&types=district%2Cplace%2Clocality


Coaldale;Six Mile Run Post Office (historical), Pennsylvania
nothing returned!
Weed, California
Edgewood, Maryland
Sylvester, West Virginia
Garrison, North Dakota
Montezuma, Georgia
Virgie, Kentucky
Onalaska, Texas
Golden, Illinois
Ruleville, Mississippi
Attica, New York
Hansville, Washington
Grosvenor Dale, Connecticut
Harpersville, Alabama
Lamont, Florida
Langdon, North Dakota
Paoli, Pennsylvania
Grawn, Michigan
Granby, Missouri
Kelleyland, Maine
Stoddard, Wisconsin
Mabel, Minnesota
Aberdeen, Idaho
Abbeville, South Carolina
Monmouth Beach, New Jersey
Sandy Spring, Maryland
Hellertown, Pennsylvania
Oconto Falls, Wisconsin
Delta Junction, Alaska
Roxbury, Vermont
Florence Hill, New York
Creighton, Missouri
Glenvil, Nebraska
Forest City, Missouri
Jackson, Minnesota
Mineral Wells, West Virginia
Sacred Heart, Minnesota
South Fulton, Tennessee
Maryland, New York
Bridgewater, Virginia
Percy, Illinois
Cape Fair, Missouri
Gardiner, New York
Franks Corner, Ohio
Malaga, Washington
Syracuse, Nebr

Example of geocoding a single location str

In [10]:
token = get_token(0)
g = geocoder.mapbox('Lagol, California', key=token,country='US',types='district,place,locality')
g = g.json
for k in g.keys():
    print(k,g[k])

address California, Pennsylvania, United States
bbox {'northeast': [40.070266991, -79.878935001], 'southwest': [40.03764208, -79.905647565]}
confidence 7
country United States
lat 40.0658
lng -79.8918
ok True
quality 0.5
raw {'id': 'place.7201990432738010', 'type': 'Feature', 'place_type': ['place'], 'relevance': 0.5, 'properties': {'wikidata': 'Q1134176'}, 'text': 'California', 'place_name': 'California, Pennsylvania, United States', 'bbox': [-79.905647565, 40.03764208, -79.878935001, 40.070266991], 'center': [-79.8918, 40.0658], 'geometry': {'type': 'Point', 'coordinates': [-79.8918, 40.0658]}, 'context': [{'id': 'district.9092829805246050', 'wikidata': 'Q497200', 'text': 'Washington County'}, {'id': 'region.13761801799111630', 'wikidata': 'Q1400', 'short_code': 'US-PA', 'text': 'Pennsylvania'}, {'id': 'country.19678805456372290', 'wikidata': 'Q30', 'short_code': 'us', 'text': 'United States'}], 'district': 'Washington County', 'region': 'Pennsylvania', 'country': 'United States'}
st